# Mixed-Integer Linear Programming Use Case: Automatic Search of Meet-in-the-Middle Preimage Attacks on AES-like Hashing

## Problem description

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_02.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_03.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_04.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_05.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_06.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_07.png" alt="MITM_Intro" width="800"/></p>

---
<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_08.png" alt="MITM_Intro" width="800"/></p>

---

## Mathematical optimization 

Mathematical optimization, which is also known as mathematical programming, is 
- the *selection* of a *best* element, with regard to some *criterion*, from some set of available alternatives.

- a declarative approach where the modeler formulates an optimization problem that captures the key features of a complex decision problem.

A mathematical optimization model has five components:

* Sets
* Parameters

* Decision variables
* Constraints
* Objective function(s)

### Mixed-Integer Linear Programming

<p align="center"><img src="MILP_Intro.png" alt="MILP_Intro" width="700"/></p>

Whereas the *simplex* method is effective for solving *linear programs*, there is no single technique for solving **integer programs**. Instead, a number of procedures have been developed, and the performance of any particular techniques appears to be highly problem-dependent. Methods to date can be classified broadly as following one of three approaches:

* enumeration techniques, including the branch-and-bound procedure;

* cutting-plane techniques; and

* group-theoretic techniques.

The **Gurobi Optimizer**:
implemented many of these methods, and provides API to call them to solves the mathematical optimization problems using state-of-the-art mathematics and computer science.
* C, C++, Java, .NET, Python, MATLAB, R API

* Callback, Solution Pool, Multiple Objectives, Multiple Scenarios, Batch Optimization...

---

In [10]:
#%pip install gurobipy

In [11]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [12]:
# Parameters
NRow = 4
NCol = 4
NMat = NRow * NCol
N_Br = NRow + 1     # 5
N_IO = 2 * NRow     # 8
WLastMC = 0 # with last mixcolumn

TR = 7   # total round
ini_r = 4   # initial round, ini_r in {0,1,2,...,TR-1}
mat_r = 1   # meet round, mat_r in {0,1,2,...,TR-1}, ini_r != mat_r
F_r = []    # forward
B_r = []    # backward
if ini_r < mat_r:
    F_r = list(range(ini_r, mat_r))
    B_r = list(range(mat_r + 1, TR)) + list(range(0, ini_r))
else: 
    B_r = list(range(mat_r + 1, ini_r))
    F_r = list(range(ini_r, TR)) + list(range(0, mat_r))

modelName = 'model_%d_x_%d_%dR_int_r%d_mat_r%d' % (NRow, NCol, TR, ini_r, mat_r)
# Declare and initialize model
m = gp.Model(modelName)
modelName

'model_4_x_4_7R_int_r4_mat_r1'

<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_09.png" alt="MITM_Intro" width="800"/></p>

---

## States

$$\mathrm{SB}_{0} \xRightarrow{\mathrm{SR}} \mathrm{MC}_{0} \xRightarrow{\mathrm{MC}} \mathrm{SB}_{1} \xRightarrow{\mathrm{SR}} \mathrm{MC}_{1} \xRightarrow{\mathrm{MC}} \cdots \mathrm{SB}_{r-1} \xRightarrow{\mathrm{SR}} \mathrm{MC}_{r-1}$$

Decision Variables Byte Order

$$
S[0,0]\quad S[0,1]\quad S[0,2]\quad S[0,3] \\
S[1,0]\quad S[1,1]\quad S[1,2]\quad S[1,3] \\
S[2,0]\quad S[2,1]\quad S[2,2]\quad S[2,3] \\
S[3,0]\quad S[3,1]\quad S[3,2]\quad S[3,3]
$$

In [13]:
# Enumeration: (x, y), grey iff x=y=1, white iff x=y=0
SB_x = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
SB_y = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
SB_g = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
SB_w = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
MC_x = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
MC_y = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
MC_g = np.ndarray(shape=(TR, NRow, NCol), dtype='object')
MC_w = np.ndarray(shape=(TR, NRow, NCol), dtype='object')

In [14]:
# SB
for ri in range(TR):
    for i in range(NRow):
        for j in range(NCol):
            SB_x[ri, i, j] = m.addVar(vtype=GRB.BINARY, name="SB_x" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri))
            SB_y[ri, i, j] = m.addVar(vtype=GRB.BINARY, name="SB_y" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri))
            SB_g[ri, i, j] = m.addVar(vtype=GRB.BINARY, name="SB_g" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri))
            SB_w[ri, i, j] = m.addVar(vtype=GRB.BINARY, name="SB_w" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri))

In [15]:
# MC and SB point to the same variables, since SR operation simply permutate the cells
for ri in range(TR):
    for i in range(NRow):
        for j in range(NCol):
            MC_x[ri, i, j] = SB_x[ri, i, (j + i)%NCol]
            MC_y[ri, i, j] = SB_y[ri, i, (j + i)%NCol]
            MC_g[ri, i, j] = SB_g[ri, i, (j + i)%NCol]
            MC_w[ri, i, j] = SB_w[ri, i, (j + i)%NCol]

In [16]:
m.update()
def printStateVarsName(aVars):
    nstr = ''
    for i in range(NRow):
        for j in range(NCol):
            nstr += aVars[i,j].VarName + ', '
        nstr += "\n"
    nstr += "\n"
    print(nstr)

print("SB_x"); printStateVarsName(SB_x[0]); print("MC_x"); printStateVarsName(MC_x[0])

SB_x
SB_x[0,0]_r[0], SB_x[0,1]_r[0], SB_x[0,2]_r[0], SB_x[0,3]_r[0], 
SB_x[1,0]_r[0], SB_x[1,1]_r[0], SB_x[1,2]_r[0], SB_x[1,3]_r[0], 
SB_x[2,0]_r[0], SB_x[2,1]_r[0], SB_x[2,2]_r[0], SB_x[2,3]_r[0], 
SB_x[3,0]_r[0], SB_x[3,1]_r[0], SB_x[3,2]_r[0], SB_x[3,3]_r[0], 


MC_x
SB_x[0,0]_r[0], SB_x[0,1]_r[0], SB_x[0,2]_r[0], SB_x[0,3]_r[0], 
SB_x[1,1]_r[0], SB_x[1,2]_r[0], SB_x[1,3]_r[0], SB_x[1,0]_r[0], 
SB_x[2,2]_r[0], SB_x[2,3]_r[0], SB_x[2,0]_r[0], SB_x[2,1]_r[0], 
SB_x[3,3]_r[0], SB_x[3,0]_r[0], SB_x[3,1]_r[0], SB_x[3,2]_r[0], 




In [17]:
DoF_init_BL = np.ndarray(shape=(NRow, NCol), dtype='object')    # blue
DoF_init_RD = np.ndarray(shape=(NRow, NCol), dtype='object')    # red
for i in range(NRow):
    for j in range(NCol):
        DoF_init_BL[i, j] = m.addVar(vtype=GRB.BINARY, name="DoF_init_BL" + ('[%d,%d]' % (i,j)))
        DoF_init_RD[i, j] = m.addVar(vtype=GRB.BINARY, name="DoF_init_RD" + ('[%d,%d]' % (i,j)))

<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_10.png" alt="MITM_Intro" width="800"/></p>

---

In [18]:
# CD: degree of freedom consumed, x for forward, y for backward
CD_x = np.ndarray(shape=(TR, NCol), dtype='object')
CD_y = np.ndarray(shape=(TR, NCol), dtype='object')
for ri in range(TR):
    for j in range(NCol):
        CD_x[ri, j] = m.addVar(lb=0, ub=NRow, vtype=GRB.INTEGER, name="CD_x_c" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        CD_y[ri, j] = m.addVar(lb=0, ub=NRow, vtype=GRB.INTEGER, name="CD_y_c" + ('[%d]' % j) + "_r" + ('[%d]' % ri))

# Intermediate values for computations on DoM
MT_n = np.empty(shape=(NCol), dtype='object')
MT_m = np.empty(shape=(NCol), dtype='object')
for j in range(NCol):
    MT_n[j] = m.addVar(lb=-NRow, ub=NRow, name="MT_n_c" + ('[%d]' % j))
    MT_m[j] = m.addVar(lb=0, ub=NRow, vtype=GRB.INTEGER, name="MT_m_c" + ('[%d]' % j))

In [19]:
DoF_BL = m.addVar(lb=1, vtype=GRB.INTEGER, name="DoF_BL")
DoF_RD = m.addVar(lb=1, vtype=GRB.INTEGER, name="DoF_RD")
DoM = m.addVar(lb=1, vtype=GRB.INTEGER, name="DoM")
Obj = m.addVar(lb=1, vtype=GRB.INTEGER, name="Obj")

In [20]:
def setObjective():
    m.addConstr(DoF_BL - gp.quicksum(DoF_init_BL.flatten()) + gp.quicksum(CD_x.flatten()) == 0)
    m.addConstr(DoF_RD - gp.quicksum(DoF_init_RD.flatten()) + gp.quicksum(CD_y.flatten()) == 0)
    m.addConstr(DoM - gp.quicksum(MT_m.flatten()) == 0)
    m.addConstr(Obj - DoF_BL <= 0)
    m.addConstr(Obj - DoF_RD <= 0)
    m.addConstr(Obj - DoM <= 0)
    m.setObjective(Obj, GRB.MAXIMIZE)

<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_11.png" alt="MITM_Intro" width="800"/></p>

---

<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_12.png" alt="MITM_Intro" width="800"/></p>

---

In [21]:
# Define indicator variables
# at column level, v==X, w==Y, mu==W
SB_X = np.ndarray(shape=(TR, NCol), dtype='object')
SB_Y = np.ndarray(shape=(TR, NCol), dtype='object')
SB_W = np.ndarray(shape=(TR, NCol), dtype='object')
MC_X = np.ndarray(shape=(TR, NCol), dtype='object')
MC_Y = np.ndarray(shape=(TR, NCol), dtype='object')
MC_W = np.ndarray(shape=(TR, NCol), dtype='object')

for ri in range(TR):
    for j in range(NCol):
        SB_X[ri, j] = m.addVar(vtype=GRB.BINARY, name="SB_Xc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        SB_Y[ri, j] = m.addVar(vtype=GRB.BINARY, name="SB_Yc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        SB_W[ri, j] = m.addVar(vtype=GRB.BINARY, name="SB_Wc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        MC_X[ri, j] = m.addVar(vtype=GRB.BINARY, name="MC_Xc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        MC_Y[ri, j] = m.addVar(vtype=GRB.BINARY, name="MC_Yc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))
        MC_W[ri, j] = m.addVar(vtype=GRB.BINARY, name="MC_Wc" + ('[%d]' % j) + "_r" + ('[%d]' % ri))

for ri in range(TR):
    for i in range(NRow):
        for j in range(NCol):
            m.addConstr(SB_g[ri, i, j] == gp.and_(SB_x[ri, i, j], SB_y[ri, i, j]))  # grey cells, and(x, y)
            m.addConstr(SB_w[ri, i, j] + SB_x[ri, i, j] + SB_y[ri, i, j] - SB_g[ri, i, j] == 1)    # white cells

for ri in range(TR):
    for j in range(NCol):
        m.addConstr(SB_X[ri, j] == gp.min_(SB_x[ri, : ,j].tolist()))    # if all x are 1 (blue or grey, no white nor red), X is given 1
        m.addConstr(SB_Y[ri, j] == gp.min_(SB_y[ri, : ,j].tolist()))    # if all y are 1 (red or grey, no white nor blue), Y is given 1
        m.addConstr(SB_W[ri, j] == gp.max_(SB_w[ri, : ,j].tolist()))    # if one is white, then whole column white
        m.addConstr(MC_X[ri, j] == gp.min_(MC_x[ri, : ,j].tolist()))
        m.addConstr(MC_Y[ri, j] == gp.min_(MC_y[ri, : ,j].tolist()))
        m.addConstr(MC_W[ri, j] == gp.max_(MC_w[ri, : ,j].tolist()))

In [39]:
type(SB_x[0, : ,0])

numpy.ndarray

### General Constraints

$$
g_i = \mathrm{and}(x_i, y_i) \text{ can also be manually formulated as } 
\begin{cases}
x_i + y_i - g_i \leq 1 \\
x_i + y_i - 2 g_i \geq 0
\end{cases}
$$
More generally
$$
g = \mathrm{and}(x_1, x_2, \ldots, x_m) \text{ can also be manually formulated as } 
\begin{cases}
\sum_{i=1}^{m}x_i - g \leq m - 1 \\
\sum_{i=1}^{m}x_i - m \cdot g \geq 0
\end{cases}
$$

For binary arrays
$$
\bold{w} = \max_{i=0}^{\mathrm{N_{Row}}-1}(w_i^I) = \mathrm{or}_{i=0}^{\mathrm{N_{Row}}-1}(w_i^I)
\text{ can also be manually formulated by linear inequalities }
\begin{cases}
(\sum_{i=0}^{\mathrm{N_{Row}}-1} w_i^I) - \mathrm{N_{Row}} \cdot \bold{w} \leq 0 \\
(\sum_{i=0}^{\mathrm{N_{Row}}-1} w_i^I) - \bold{w} \geq 0
\end{cases}
$$

$$
\bold{x} = \min_{i=0}^{\mathrm{N_{Row}}-1}(x_i^I) = \mathrm{and}_{i=0}^{\mathrm{N_{Row}}-1}(x_i^I) 
\text{ can also be manually formulated by linear inequalities }
\begin{cases}
(\sum_{i=0}^{\mathrm{N_{Row}}-1} x_i^I) - \mathrm{N_{Row}} \cdot \bold{x} \geq 0 \\
(\sum_{i=0}^{\mathrm{N_{Row}}-1} x_i^I) - \bold{x} \leq \mathrm{N_{Row}} - 1
\end{cases}
$$


Gurobi API supports some fundamental constraints of MIP.
* simple constraints (max, min, abs, and, or, norm, indicator, piecewise-linear)
* function constraints (polynomial, $y = e^x$, $y=a^x$, $y=ln(x)$, $y=x^a$, ...)


In [22]:
def MC_RULE(
    MC_I_Col_x,
    MC_I_Col_y,
    MC_I_Col_X,
    MC_I_Col_Y,
    MC_I_Col_W,
    MC_O_Col_x,
    MC_O_Col_y,
    CD_Col_x,
    CD_Col_y):
    # Introduce 0-1 indicator variables for each input column
    # Constraints for defining attribute-propagation through MC
    m.addConstr(gp.quicksum(MC_O_Col_x) + NRow * MC_I_Col_W <= NRow)
    m.addConstr(gp.quicksum(MC_O_Col_x) + gp.quicksum(MC_I_Col_x) - N_Br * MC_I_Col_X <= N_IO - N_Br)
    m.addConstr(gp.quicksum(MC_O_Col_x) + gp.quicksum(MC_I_Col_x) - N_IO * MC_I_Col_X >= 0)
    
    m.addConstr(gp.quicksum(MC_O_Col_y) + NRow * MC_I_Col_W <= NRow)
    m.addConstr(gp.quicksum(MC_O_Col_y) + gp.quicksum(MC_I_Col_y) - N_Br * MC_I_Col_Y <= N_IO - N_Br)
    m.addConstr(gp.quicksum(MC_O_Col_y) + gp.quicksum(MC_I_Col_y) - N_IO * MC_I_Col_Y >= 0)
    # Constraints for canceling impact by consuming DoF
    m.addConstr(gp.quicksum(MC_O_Col_x) - NRow * MC_I_Col_X - CD_Col_y == 0)
    m.addConstr(gp.quicksum(MC_O_Col_y) - NRow * MC_I_Col_Y - CD_Col_x == 0)

$\mathrm{N_{IO}} - \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)}$ equals the number of active cells impacted by RED

* intrinsic bound
    $$0 \leq \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \leq \mathrm{N_{IO}}$$

* If $\bold{x} = 1$:
    $$\forall x_i^O = 1$$
    $$\sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} = \mathrm{N_{IO}}$$
    $$\boxed{\mathrm{N_{IO}} \leq} \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \leq \mathrm{N_{IO}}$$

* If $\bold{x} = 0$:
    $$\mathrm{N_{IO}} - \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \geq \mathrm{N_{Br}}$$
    $$\sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \boxed{\leq \mathrm{N_{IO}} - \mathrm{N_{Br}}}$$

* Formulated using **constraint feasibility**
    $$\mathrm{N_{IO}} \boxed{- B_1 \cdot (1 -  \bold{x})} \leq \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \leq \mathrm{N_{IO}} - \mathrm{N_{Br}} \boxed{+ B_2 \cdot \bold{x}} $$

* For any $\bold{x}$
    $$ \boxed{B_1 = \mathrm{N_{IO}}}, \quad \boxed{B_2 = \mathrm{N_{Br}}}$$
    $$\mathrm{N_{IO}} \cdot \bold{x} \leq \sum_{i=0}^{\mathrm{N_{Row}}-1}{(x_i^O + x_i^I)} \leq \mathrm{N_{IO}} - \mathrm{N_{Br}} + \mathrm{N_{Br}} \cdot \bold{x} $$


***Remark: Logical Constraints***

* **Constraint Feasibility**

Possibly the simplest logical question that can be asked in mathematical programming is whether a given choice of the decision variables satisfies a constraint. More precisely, *when* is the general constraint
$$
\rightarrow f(x_1, x_2, \ldots, x_n) \leq b
$$
satisfied?

We introduce a binary variable $y$ with the interpretation:
$$
y = 
 \begin{cases} 0 & \text{ if the constraint is known to be satisfied} \\
 1 & \text{ otherwise }
 \end{cases}
$$
and write
$$
\boxed{f(x_1, x_2, \ldots, x_n) - By \leq b},
$$
where the constant $B$ is chosen to be **large enough** so that the constraint always is satisfied if $y=1$; that is $f(x_1, x_2, \ldots, x_n) \leq b + B$ for every possible choice of the decision variables $x_1, x_2,\ldots, x_n$ at our disposal. Whenever $y=0$ gives a feasible solution to constraint $f(\cdot) \leq b$, we know that $f(\cdot) - By \leq b$ must be satisfied.

In practice, it is usually very easy to determine a large number to serve as $B$, although generally it is best to use the smallest possible value of $B$ in order to avoid numerical difficulties during computations.

Similarly, for 
$$
f(x_1, x_2, \ldots, x_n) \geq b,
$$
we write
write
$$
\boxed{f(x_1, x_2, \ldots, x_n) + By \geq b},
$$
where the constant $B$ is chosen to be **large enough** so that the constraint always is satisfied if $y=1$.


<p align="center"><img src="Intro_MitM/00_MITM_AES_Hashing_Short_Program_Page_14.png" alt="MITM_Intro" width="800"/></p>

---

In [23]:
def Match_RULE(
    MC_I_Col_x,
    MC_I_Col_y,
    MC_I_Col_g,
    MC_O_Col_x,
    MC_O_Col_y,
    MC_O_Col_g,
    MT_Col_n,
    MT_Col_m):
    m.addConstr(MT_Col_n == 
        gp.quicksum(MC_I_Col_x) + gp.quicksum(MC_I_Col_y) - gp.quicksum(MC_I_Col_g) + 
        gp.quicksum(MC_O_Col_x) + gp.quicksum(MC_O_Col_y) - gp.quicksum(MC_O_Col_g) - NRow)
    m.addConstr(MT_Col_m == gp.max_(MT_Col_n, 0))

In [24]:
def addConstrs_Start_Round():
    # Starting round do not allow unknown
    for i in range(NRow):
        for j in range(NCol):
            m.addConstr(SB_x[ini_r, i, j] + SB_y[ini_r, i, j] >= 1)
            m.addConstr(DoF_init_BL[i, j] + SB_y[ini_r, i, j] == 1)
            m.addConstr(DoF_init_RD[i, j] + SB_x[ini_r, i, j] == 1)

In [25]:
def addConstrs_Inter_Round(ri):
    if WLastMC == 0 and ri == TR - 1:
        for j in range(NCol):
            m.addConstr(CD_x[ri,j] == 0)
            m.addConstr(CD_y[ri,j] == 0)
            for i in range(NRow):
                m.addConstr(MC_x[ri, i, j] - SB_x[(ri + 1) % TR, i, j] == 0)
                m.addConstr(MC_y[ri, i, j] - SB_y[(ri + 1) % TR, i, j] == 0)
    elif ri in F_r:
        for ci in range(NCol):
            MC_RULE(
                MC_x[ri, :, ci],
                MC_y[ri, :, ci],
                MC_X[ri, ci],
                MC_Y[ri, ci],
                MC_W[ri, ci],
                SB_x[(ri + 1) % TR, :, ci],
                SB_y[(ri + 1) % TR, :, ci],
                CD_x[ri,ci],
                CD_y[ri,ci]
            )
    elif ri in B_r:
        for ci in range(NCol):
            MC_RULE(
                SB_x[(ri + 1) % TR, :, ci],
                SB_y[(ri + 1) % TR, :, ci],
                SB_X[(ri + 1) % TR, ci],
                SB_Y[(ri + 1) % TR, ci],
                SB_W[(ri + 1) % TR, ci],
                MC_x[ri, :, ci],
                MC_y[ri, :, ci],
                CD_x[ri,ci],
                CD_y[ri,ci]
            )
    else:
        print('Should not reach here: addConstrs_Inter_Round():: ri = %d' % ri)


In [26]:
def addConstrs_Match_Round():
    if WLastMC == 0 and mat_r == TR - 1:
        MT_w = np.ndarray(shape=(NRow, NCol), dtype='object')
        for i in range(NRow):
            for j in range(NCol):
                MT_w[i, j] = m.addVar(vtype=GRB.BINARY, name="MT_w" + ('[%d,%d]' % (i, j)))
                m.addConstr(MT_w[i, j] == gp.or_(MC_w[mat_r, i, j], SB_w[(mat_r + 1)%TR, i, j]))
        for j in range(NCol):
            m.addConstr(MT_m[j] + gp.quicksum(MT_w[:, j]) == NRow)
            m.addConstr(CD_x[mat_r, j] == 0)
            m.addConstr(CD_y[mat_r, j] == 0)
    else:
        for ci in range(NCol):
            Match_RULE(
                MC_x[mat_r, :, ci],
                MC_y[mat_r, :, ci],
                MC_g[mat_r, :, ci],
                SB_x[(mat_r + 1) % TR, :, ci],
                SB_y[(mat_r + 1) % TR, :, ci],
                SB_g[(mat_r + 1) % TR, :, ci],
                MT_n[ci],
                MT_m[ci]
            )
            m.addConstr(CD_x[mat_r, ci] == 0)
            m.addConstr(CD_y[mat_r, ci] == 0)

In [27]:
setObjective()
for ri in range(TR):
    if ri == ini_r:
        addConstrs_Start_Round()
    if ri == mat_r:
        addConstrs_Match_Round()
    else:
        addConstrs_Inter_Round(ri)

m.write(m.modelName + '.lp' )

In [28]:
def writeSol():
    if m.SolCount > 0:
        if m.getParamInfo(GRB.Param.PoolSearchMode)[2] > 0:
            gv = m.getVars()
            names = m.getAttr('VarName', gv)
            for i in range(m.SolCount):
                m.params.SolutionNumber = i
                xn = m.getAttr('Xn', gv)
                lines = ["{} {}".format(v1, v2) for v1, v2 in zip(names, xn)]
                with open('{}_{}.sol'.format(m.modelName, i), 'w') as f:
                    f.write("# Solution for model {}\n".format(m.modelName))
                    f.write("# Objective value = {}\n".format(m.PoolObjVal))
                    f.write("\n".join(lines))
        else:
            m.write(m.modelName + '.sol')
    else:
        print('infeasible')

In [29]:
#m.setParam(GRB.Param.PoolSearchMode, 2)
#m.setParam(GRB.Param.PoolSolutions,  5)
#m.setParam(GRB.Param.Threads, 8)
#m.setParam(GRB.Param.SolFiles, modelName)

m.optimize()
writeSol()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 378 rows, 716 columns and 1985 nonzeros
Model fingerprint: 0x1037c3d9
Model has 284 general constraints
Variable types: 4 continuous, 712 integer (648 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 4e+00]
Presolve added 245 rows and 0 columns
Presolve removed 0 rows and 291 columns
Presolve time: 0.02s
Presolved: 623 rows, 425 columns, 2239 nonzeros
Variable types: 0 continuous, 425 integer (376 binary)

Root relaxation: objective 8.000000e+00, 469 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.00000    0  132          -    8.00000      -     -    0s
  

In [30]:
def drawSol(outfile=None):
    if outfile == None:
        outfile = m.modelName + '.sol'
    solFile = open(outfile, 'r')
    Sol = dict()
    for line in solFile:
        if line[0] != '#':
            temp = line
            temp = temp.split()
            Sol[temp[0]] = round(float(temp[1]))
    SB_x_v = np.ndarray(shape=(TR, NRow, NCol), dtype='int')
    SB_y_v = np.ndarray(shape=(TR, NRow, NCol), dtype='int')
    MC_x_v = np.ndarray(shape=(TR, NRow, NCol), dtype='int')
    MC_y_v = np.ndarray(shape=(TR, NRow, NCol), dtype='int')
    DoF_init_BL_v = np.ndarray(shape=(NRow, NCol), dtype='int')
    DoF_init_RD_v = np.ndarray(shape=(NRow, NCol), dtype='int')
    CD_x_v = np.ndarray(shape=(TR, NCol), dtype='int')
    CD_y_v = np.ndarray(shape=(TR, NCol), dtype='int')
    for ri in range(TR):
        for i in range(NRow):
            for j in range(NCol):
                SB_x_v[ri, i, j] = Sol["SB_x" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri)]
                SB_y_v[ri, i, j] = Sol["SB_y" + ('[%d,%d]' % (i, j)) + "_r" + ('[%d]' % ri)]
    for ri in range(TR):
        for i in range(NRow):
            for j in range(NCol):
                MC_x_v[ri, i, j] = SB_x_v[ri, i, (j + i)%NCol]
                MC_y_v[ri, i, j] = SB_y_v[ri, i, (j + i)%NCol]
    for i in range(NRow):
        for j in range(NCol):
            DoF_init_BL_v[i, j] = Sol["DoF_init_BL" + ('[%d,%d]' % (i,j))]
            DoF_init_RD_v[i, j] = Sol["DoF_init_RD" + ('[%d,%d]' % (i,j))]
    for ri in range(TR):
        for j in range(NCol):
            CD_x_v[ri, j] = Sol["CD_x_c" + ('[%d]' % j) + "_r" + ('[%d]' % ri)]
            CD_y_v[ri, j] = Sol["CD_y_c" + ('[%d]' % j) + "_r" + ('[%d]' % ri)]
    DoF_BL_v = Sol["DoF_BL"]
    DoF_RD_v = Sol["DoF_RD"]
    DoM_v = Sol["DoM"]

    CM = np.ndarray(shape=(2, 2),dtype='object')
    CM[0, 0] = '\\fill[\\UW]'
    CM[0, 1] = '\\fill[\\BW]'
    CM[1, 0] = '\\fill[\\FW]'
    CM[1, 1] = '\\fill[\\CW]'
    if NRow == 4:
        HO = NRow
        WO = NCol
    else:
        HO = NRow
        WO = NCol // 2
    ini_d1 = 0
    ini_d2 = 0
    for i in range(NRow):
        for j in range(NCol):
            ini_d1 += DoF_init_BL_v[i,j]
            ini_d2 += DoF_init_RD_v[i,j]
    fid = open(outfile + '.tex', 'w')
    fid.write(
        '\\documentclass{standalone}' + '\n'
        '\\usepackage[usenames,dvipsnames]{xcolor}' + '\n'
        '\\usepackage{amsmath,amssymb,mathtools}' + '\n'
        '\\usepackage{tikz,calc,pgffor}' + '\n'
        '\\usepackage{xspace}' + '\n'
        '\\usetikzlibrary{crypto.symbols,patterns,calc}' + '\n'
        '\\tikzset{shadows=no}' + '\n'
        '\\input{macro}' + '\n')
    fid.write('\n\n')
    fid.write(
        '\\begin{document}' + '\n' +
        '\\begin{tikzpicture}[scale=0.2, every node/.style={font=\\boldmath\\bf}]' + '\n'
	    '\\everymath{\\scriptstyle}' + '\n'
	    '\\tikzset{edge/.style=->, >=stealth, arrow head=8pt, thick};' + '\n')
    fid.write('\n\n')

    for r in range(TR):
        CD_BL = 0
        CD_RD = 0
        for i in range(NCol):
            CD_BL += CD_x_v[r, i]
            CD_RD += CD_y_v[r, i]
        O = 0
        ## SB
        fid.write('\\begin{scope}[yshift =' + str(- r * (NRow + HO))+' cm, xshift =' + str(O * (NCol + WO))+' cm]'+'\n')
        for i in range(NRow):
            row = NRow - 1 - i
            for j in range(NCol):
                col = j
                fid.write(CM[SB_x_v[r,i,j], SB_y_v[r,i,j]] + ' ('+str(col)+','+str(row)+') rectangle +(1,1);'+'\n')
        fid.write('\\draw (0,0) rectangle (' + str(NCol) + ',' + str(NRow) + ');' + '\n')
        for i in range(1, NRow):
            fid.write('\\draw (' + str(0) + ',' + str(i) + ') rectangle (' + str(NCol) + ',' + str(0) + ');' + '\n')
        for i in range(1, NCol):
            fid.write('\\draw (' + str(i) + ',' + str(0) + ') rectangle (' + str(0) + ',' + str(NRow) + ');' + '\n')
        fid.write('\\path (' + str(NCol//2) + ',' + str(NRow + 0.5) + ') node {\\scriptsize$\\SB^' + str(r) + '$};'+'\n')
        if r in B_r:
            fid.write('\\draw[edge, <-] (' + str(NCol) + ',' + str(NRow//2) + ') -- node[above] {\\tiny SB} node[below] {\\tiny SR} +(' + str(WO) + ',' + '0);' + '\n')
        else:
            fid.write('\\draw[edge, ->] (' + str(NCol) + ',' + str(NRow//2) + ') -- node[above] {\\tiny SB} node[below] {\\tiny SR} +(' + str(WO) + ',' + '0);' + '\n')
        if r == ini_r:
            fid.write('\\path (' + str(NCol//2) + ',' + str(-0.8) + ') node {\\scriptsize$(+' + str(ini_d1) + '~\\DoFF,~+' + str(ini_d2) + '~\\DoFB)$};'+'\n')
            fid.write('\\path (' + str(-2) + ',' + str(0.8) + ') node {\\scriptsize$\\StENC$};'+'\n')
        fid.write('\n'+'\\end{scope}'+'\n')
        fid.write('\n\n')

        O = O + 1
        ## MC
        fid.write('\\begin{scope}[yshift =' + str(- r * (NRow + HO))+' cm, xshift =' +str(O * (NCol + WO))+' cm]'+'\n')
        for i in range(NRow):
            row = NRow - 1 - i
            for j in range(NCol):
                col = j
                fid.write(CM[MC_x_v[r,i,j], MC_y_v[r,i,j]] + ' ('+str(col)+','+str(row)+') rectangle +(1,1);'+'\n')
        fid.write('\\draw (0,0) rectangle (' + str(NCol) + ',' + str(NRow) + ');' + '\n')
        for i in range(1, NRow):
            fid.write('\\draw (' + str(0) + ',' + str(i) + ') rectangle (' + str(NCol) + ',' + str(0) + ');' + '\n')
        for i in range(1, NCol):
            fid.write('\\draw (' + str(i) + ',' + str(0) + ') rectangle (' + str(0) + ',' + str(NRow) + ');' + '\n')
        fid.write('\\path (' + str(NCol//2) + ',' + str(NRow + 0.5) + ') node {\\scriptsize$\\MC^' + str(r) + '$};'+'\n')
        op = 'MC'
        if r == TR - 1 and WLastMC == 0:
            op = 'I'
        if r in B_r:
            fid.write('\\draw[edge, <-] (' + str(NCol) + ',' + str(NRow//2) + ') -- node[above] {\\tiny ' + op + '} +(' + str(WO) + ',' + '0);' + '\n')
        if r in F_r:
            fid.write('\\draw[edge, ->] (' + str(NCol) + ',' + str(NRow//2) + ') -- node[above] {\\tiny ' + op + '} +(' + str(WO) + ',' + '0);' + '\n')
        if r == mat_r:
            fid.write('\\draw[edge, -] (' + str(NCol) + ',' + str(NRow//2) + ') -- node[above] {\\tiny ' + op + '} +(' + str(WO) + ',' + '0);' + '\n')
            fid.write('\\draw[edge, ->] (' + str(NCol) + ',' + str(NRow//2) + ') --  +(' + str(WO//2) + ',' + '0);' + '\n')
            fid.write('\\draw[edge, ->] (' + str(NCol + WO) + ',' + str(NRow//2) + ') --  +(' + str(-WO//2) + ',' + '0);' + '\n')
    
            fid.write('\\path (' + str(NCol + WO//2) + ',' + str(-0.8) + ') node {\\scriptsize Match};' + '\n')
            fid.write('\\path (' + str(-2) + ',' + str(0.1) + ') node {\\scriptsize$\\EndFwd$};' + '\n')
            fid.write('\\path (' + str(NCol + WO + NCol + 2) + ',' + str(0.1) + ') node {\\scriptsize$\\EndBwd$};' + '\n')
        else:
            fid.write('\\path (' + str((NCol + WO) - WO//2) + ',' + str(-0.8) + ') node {\\scriptsize$ (-' + str(CD_BL) + '~\\DoFF,~-' + str(CD_RD) + '~\\DoFB)$};'+'\n')
        fid.write('\n'+'\\end{scope}'+'\n')
        fid.write('\n\n')

        O = O + 1
        ## SB r+1
        fid.write('\\begin{scope}[yshift =' + str(- r * (NRow + HO))+' cm, xshift =' +str(O * (NCol + WO))+' cm]'+'\n')
        for i in range(NRow):
            row = NRow - 1 - i
            for j in range(NCol):
                col = j
                fid.write(CM[SB_x_v[(r+1)%TR,i,j], SB_y_v[(r+1)%TR,i,j]] + ' ('+str(col)+','+str(row)+') rectangle +(1,1);'+'\n')
        fid.write('\\draw (0,0) rectangle (' + str(NCol) + ',' + str(NRow) + ');' + '\n')
        for i in range(1, NRow):
            fid.write('\\draw (' + str(0) + ',' + str(i) + ') rectangle (' + str(NCol) + ',' + str(0) + ');' + '\n')
        for i in range(1, NCol):
            fid.write('\\draw (' + str(i) + ',' + str(0) + ') rectangle (' + str(0) + ',' + str(NRow) + ');' + '\n')
        fid.write('\\path (' + str(NCol//2) + ',' + str(NRow + 0.5) + ') node {\\scriptsize$\\SB^' + str((r+1)%TR) + '$};'+'\n')
        fid.write('\n'+'\\end{scope}'+'\n')
        fid.write('\n\n')
    ## Final
    fid.write('\\begin{scope}[yshift =' + str(- TR * (NRow + HO) + HO)+' cm, xshift =' +str(2 * (NCol + WO))+' cm]'+'\n')
    fid.write(
        '\\node[draw, thick, rectangle, text width=6.5cm, label={[shift={(-2.8,-0)}]\\footnotesize Config}] at (-7, 0) {' + '\n'
	    '{\\footnotesize' + '\n'
	    '$\\bullet~(\\varInitBL,~\\varInitRD)~=~(+' + str(ini_d1) + '~\\DoFF,~+' + str(ini_d2) + '~\\DoFB)~$' + '\\\ \n'
	    '$\\bullet~(\\varDoFBL,~\\varDoFRD,~\\varDoM)~=~(+' + 
        str(int(DoF_BL_v)) + '~\\DoFF,~+' + 
        str(int(DoF_RD_v)) + '~\\DoFB,~+' + 
        str(int(DoM_v )) + '~\\DoM)$' + '\n'
	    '}' + '\n'
	    '};' + '\n'
        )
    fid.write('\n'+'\\end{scope}'+'\n')
    fid.write('\n\n')
    fid.write('\\end{tikzpicture}'+'\n\n'+'\\end{document}')
    fid.close()
    from os import system
    system("pdflatex -output-directory='./' ./" + outfile + ".tex") 

In [31]:
#for i in range(5):
#    fnp = m.modelName + "_%d" % (i) +'.sol'
#    drawSol(fnp)
fnp = m.modelName + '.sol'
drawSol(fnp)

sh: pdflatex: command not found


In [41]:
type(gp.quicksum(MC_x[0, :, 0]))

gurobipy.LinExpr